<a href="https://colab.research.google.com/github/ShouqAbdulrahim/-Misinformation-Detection-and-Generation-Using-Large-Language-Models-/blob/main/Bert(llmfake)_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
#

# Learning rate : 1e-5 Lora : 64


In [ ]:
from transformers import EarlyStoppingCallback, AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import torch
from torch.nn.functional import softmax

import pandas as pd
from transformers import AutoTokenizer
from datasets import Dataset

train_df = pd.read_csv("llmfake_mergedTrain.csv")
test_df = pd.read_csv("llmfake_mergedTest.csv")

def clean_df(df):
   return df.rename(columns={"synthetic_misinformation": "text"})[["text", "label"]]

train_df = clean_df(train_df)
test_df = clean_df(test_df)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_valid = train_dataset.train_test_split(test_size=0.2, seed=42)

train_split = train_valid['train']
valid_split = train_valid['test']

base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=64,
    lora_dropout=0.1
)

model = get_peft_model(base_model, lora_config)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary', zero_division=0)
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

training_args = TrainingArguments(
    output_dir="./bert_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    num_train_epochs=20,
    weight_decay=0.01,
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    logging_steps=10
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_split,
    eval_dataset=valid_split,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


trainer.train()

trainer.evaluate(test_dataset)

test_texts = [
    "The economy is improving according to alien scientists",
     "King Salman is not the King of Saudi Arabia.",#true
     "COVID-19 vaccines have been approved by the World Health Organization", #true
    "Aliens landed in New York and started dancing on Broadway.",
    "	The moon is made entirely of cheese",
    "The Eiffel Tower is located in Paris, France"#true
]

inputs = tokenizer(test_texts, return_tensors="pt", padding=True, truncation=True).to(model.device)
with torch.no_grad():
    outputs = model(**inputs)
    probs = softmax(outputs.logits, dim=1)
    preds = torch.argmax(probs, dim=1)

for text, pred in zip(test_texts, preds):
    print(f"Text: {text}")
    print("Prediction:", "True News ✅" if pred == 1 else "Misinformation ❌")
    print("-" * 60)


final_results = trainer.evaluate(test_dataset)
print("\n📊 Final Evaluation Results on Test Set:")
for key, value in final_results.items():
    print(f"{key}: {value:.4f}")


Map:   0%|          | 0/5336 [00:00<?, ? examples/s]

Map:   0%|          | 0/1336 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.684800,0.682246,0.607678,0.608621,0.647706,0.627556
2,0.652400,0.633476,0.679775,0.658842,0.772477,0.711149
3,0.558000,0.593294,0.724719,0.698891,0.809174,0.750000
4,0.525900,0.543280,0.750000,0.743007,0.779817,0.760967
5,0.507400,0.513822,0.764045,0.765880,0.774312,0.770073
6,0.466100,0.501787,0.775281,0.762478,0.812844,0.786856
7,0.490700,0.476487,0.792135,0.790991,0.805505,0.798182
8,0.442300,0.459367,0.796816,0.794964,0.811009,0.802906
9,0.470000,0.449288,0.799625,0.790861,0.825688,0.807899
10,0.444500,0.427235,0.805243,0.808044,0.811009,0.809524


Text: The economy is improving according to alien scientists
Prediction: Misinformation ❌
------------------------------------------------------------
Text: King Salman is not the King of Saudi Arabia.
Prediction: Misinformation ❌
------------------------------------------------------------
Text: COVID-19 vaccines have been approved by the World Health Organization
Prediction: Misinformation ❌
------------------------------------------------------------
Text: Aliens landed in New York and started dancing on Broadway.
Prediction: Misinformation ❌
------------------------------------------------------------
Text: 	The moon is made entirely of cheese
Prediction: Misinformation ❌
------------------------------------------------------------
Text: The Eiffel Tower is located in Paris, France
Prediction: Misinformation ❌
------------------------------------------------------------

📊 Final Evaluation Results on Test Set:
eval_loss: 0.3803
eval_accuracy: 0.8331
eval_precision: 0.8133
eval_reca

# Learning rate : 2e-5 Lora : 64


In [ ]:
from transformers import EarlyStoppingCallback, AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import torch
from torch.nn.functional import softmax

import pandas as pd
from transformers import AutoTokenizer
from datasets import Dataset

train_df = pd.read_csv("llmfake_mergedTrain.csv")
test_df = pd.read_csv("llmfake_mergedTest.csv")

def clean_df(df):
   return df.rename(columns={"synthetic_misinformation": "text"})[["text", "label"]]

train_df = clean_df(train_df)
test_df = clean_df(test_df)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_valid = train_dataset.train_test_split(test_size=0.2, seed=42)

train_split = train_valid['train']
valid_split = train_valid['test']

base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=64,
    lora_dropout=0.1
)

model = get_peft_model(base_model, lora_config)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary', zero_division=0)
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

training_args = TrainingArguments(
    output_dir="./bert_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    num_train_epochs=20,
    weight_decay=0.01,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    logging_steps=10
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_split,
    eval_dataset=valid_split,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


trainer.train()

trainer.evaluate(test_dataset)

test_texts = [
    "The economy is improving according to alien scientists",
     "King Salman is not the King of Saudi Arabia.",#true
     "COVID-19 vaccines have been approved by the World Health Organization", #true
    "Aliens landed in New York and started dancing on Broadway.",
    "	The moon is made entirely of cheese",
    "The Eiffel Tower is located in Paris, France"#true
]

inputs = tokenizer(test_texts, return_tensors="pt", padding=True, truncation=True).to(model.device)
with torch.no_grad():
    outputs = model(**inputs)
    probs = softmax(outputs.logits, dim=1)
    preds = torch.argmax(probs, dim=1)

for text, pred in zip(test_texts, preds):
    print(f"Text: {text}")
    print("Prediction:", "True News ✅" if pred == 1 else "Misinformation ❌")
    print("-" * 60)


final_results = trainer.evaluate(test_dataset)
print("\n📊 Final Evaluation Results on Test Set:")
for key, value in final_results.items():
    print(f"{key}: {value:.4f}")


Map:   0%|          | 0/5336 [00:00<?, ? examples/s]

Map:   0%|          | 0/1336 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.650600,0.620769,0.660112,0.618490,0.871560,0.723534
2,0.522100,0.484683,0.764981,0.722054,0.877064,0.792046
3,0.426500,0.398707,0.822097,0.787682,0.891743,0.836489
4,0.338800,0.358147,0.856742,0.832203,0.900917,0.865198
5,0.319700,0.336942,0.861423,0.825944,0.922936,0.871750
6,0.324600,0.318409,0.870787,0.846678,0.911927,0.878092
7,0.263700,0.303134,0.891386,0.856905,0.944954,0.898778
8,0.274000,0.289445,0.886704,0.864261,0.922936,0.892635
9,0.263900,0.294508,0.887640,0.835703,0.970642,0.898132
10,0.284300,0.257606,0.902622,0.878216,0.939450,0.907801


Text: The economy is improving according to alien scientists
Prediction: True News ✅
------------------------------------------------------------
Text: King Salman is not the King of Saudi Arabia.
Prediction: Misinformation ❌
------------------------------------------------------------
Text: COVID-19 vaccines have been approved by the World Health Organization
Prediction: True News ✅
------------------------------------------------------------
Text: Aliens landed in New York and started dancing on Broadway.
Prediction: Misinformation ❌
------------------------------------------------------------
Text: 	The moon is made entirely of cheese
Prediction: Misinformation ❌
------------------------------------------------------------
Text: The Eiffel Tower is located in Paris, France
Prediction: Misinformation ❌
------------------------------------------------------------

📊 Final Evaluation Results on Test Set:
eval_loss: 0.2028
eval_accuracy: 0.9319
eval_precision: 0.9111
eval_recall: 0.9520

# Learning rate : 3e-5 Lora : 64


In [ ]:
from transformers import EarlyStoppingCallback, AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import torch
from torch.nn.functional import softmax

import pandas as pd
from transformers import AutoTokenizer
from datasets import Dataset

train_df = pd.read_csv("llmfake_mergedTrain.csv")
test_df = pd.read_csv("llmfake_mergedTest.csv")

def clean_df(df):
   return df.rename(columns={"synthetic_misinformation": "text"})[["text", "label"]]

train_df = clean_df(train_df)
test_df = clean_df(test_df)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_valid = train_dataset.train_test_split(test_size=0.2, seed=42)

train_split = train_valid['train']
valid_split = train_valid['test']

base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=64,
    lora_dropout=0.1
)

model = get_peft_model(base_model, lora_config)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary', zero_division=0)
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

training_args = TrainingArguments(
    output_dir="./bert_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    num_train_epochs=20,
    weight_decay=0.01,
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    logging_steps=10
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_split,
    eval_dataset=valid_split,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


trainer.train()

trainer.evaluate(test_dataset)

test_texts = [
    "The economy is improving according to alien scientists",
     "King Salman is not the King of Saudi Arabia.",#true
     "COVID-19 vaccines have been approved by the World Health Organization", #true
    "Aliens landed in New York and started dancing on Broadway.",
    "	The moon is made entirely of cheese",
    "The Eiffel Tower is located in Paris, France"#true
]

inputs = tokenizer(test_texts, return_tensors="pt", padding=True, truncation=True).to(model.device)
with torch.no_grad():
    outputs = model(**inputs)
    probs = softmax(outputs.logits, dim=1)
    preds = torch.argmax(probs, dim=1)

for text, pred in zip(test_texts, preds):
    print(f"Text: {text}")
    print("Prediction:", "True News ✅" if pred == 1 else "Misinformation ❌")
    print("-" * 60)


final_results = trainer.evaluate(test_dataset)
print("\n📊 Final Evaluation Results on Test Set:")
for key, value in final_results.items():
    print(f"{key}: {value:.4f}")


Map:   0%|          | 0/5336 [00:00<?, ? examples/s]

Map:   0%|          | 0/1336 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.586800,0.557514,0.689139,0.634981,0.919266,0.751124
2,0.449400,0.403090,0.815543,0.780645,0.888073,0.830901
3,0.372800,0.346380,0.853933,0.833619,0.891743,0.861702
4,0.294300,0.310828,0.870787,0.838602,0.924771,0.879581
5,0.263300,0.291793,0.884831,0.842532,0.952294,0.894057
6,0.266600,0.263973,0.899813,0.875000,0.937615,0.905226
7,0.209400,0.240321,0.914794,0.894097,0.944954,0.918822
8,0.205700,0.230139,0.916667,0.887755,0.957798,0.921447
9,0.181500,0.249378,0.910112,0.861514,0.981651,0.917667
10,0.200900,0.192648,0.935393,0.920495,0.955963,0.937894


Text: The economy is improving according to alien scientists
Prediction: True News ✅
------------------------------------------------------------
Text: King Salman is not the King of Saudi Arabia.
Prediction: Misinformation ❌
------------------------------------------------------------
Text: COVID-19 vaccines have been approved by the World Health Organization
Prediction: True News ✅
------------------------------------------------------------
Text: Aliens landed in New York and started dancing on Broadway.
Prediction: Misinformation ❌
------------------------------------------------------------
Text: 	The moon is made entirely of cheese
Prediction: Misinformation ❌
------------------------------------------------------------
Text: The Eiffel Tower is located in Paris, France
Prediction: Misinformation ❌
------------------------------------------------------------

📊 Final Evaluation Results on Test Set:
eval_loss: 0.1469
eval_accuracy: 0.9558
eval_precision: 0.9427
eval_recall: 0.9675

# Learning rate : 5e-5 Lora : 64


In [ ]:
from transformers import EarlyStoppingCallback, AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import torch
from torch.nn.functional import softmax

import pandas as pd
from transformers import AutoTokenizer
from datasets import Dataset

train_df = pd.read_csv("llmfake_mergedTrain.csv")
test_df = pd.read_csv("llmfake_mergedTest.csv")

def clean_df(df):
   return df.rename(columns={"synthetic_misinformation": "text"})[["text", "label"]]

train_df = clean_df(train_df)
test_df = clean_df(test_df)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_valid = train_dataset.train_test_split(test_size=0.2, seed=42)

train_split = train_valid['train']
valid_split = train_valid['test']

base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=64,
    lora_dropout=0.1
)

model = get_peft_model(base_model, lora_config)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary', zero_division=0)
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

training_args = TrainingArguments(
    output_dir="./bert_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    num_train_epochs=20,
    weight_decay=0.01,
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    logging_steps=10
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_split,
    eval_dataset=valid_split,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


trainer.train()

trainer.evaluate(test_dataset)

test_texts = [
    "The economy is improving according to alien scientists",
     "King Salman is not the King of Saudi Arabia.",#true
     "COVID-19 vaccines have been approved by the World Health Organization", #true
    "Aliens landed in New York and started dancing on Broadway.",
    "	The moon is made entirely of cheese",
    "The Eiffel Tower is located in Paris, France"#true
]

inputs = tokenizer(test_texts, return_tensors="pt", padding=True, truncation=True).to(model.device)
with torch.no_grad():
    outputs = model(**inputs)
    probs = softmax(outputs.logits, dim=1)
    preds = torch.argmax(probs, dim=1)

for text, pred in zip(test_texts, preds):
    print(f"Text: {text}")
    print("Prediction:", "True News ✅" if pred == 1 else "Misinformation ❌")
    print("-" * 60)


final_results = trainer.evaluate(test_dataset)
print("\n📊 Final Evaluation Results on Test Set:")
for key, value in final_results.items():
    print(f"{key}: {value:.4f}")


Map:   0%|          | 0/5336 [00:00<?, ? examples/s]

Map:   0%|          | 0/1336 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.492300,0.466733,0.765918,0.713459,0.904587,0.797735
2,0.388000,0.347706,0.852996,0.814935,0.921101,0.864772
3,0.307600,0.290720,0.879213,0.845515,0.933945,0.887533
4,0.210100,0.244332,0.911985,0.886792,0.948624,0.916667
5,0.195500,0.217506,0.922285,0.894198,0.961468,0.926614
6,0.179500,0.198615,0.929775,0.906574,0.961468,0.933215
7,0.144500,0.182060,0.936330,0.923623,0.954128,0.938628
8,0.117800,0.162439,0.941948,0.918544,0.972477,0.944742
9,0.097700,0.151583,0.948502,0.926829,0.976147,0.950849
10,0.132100,0.136373,0.956929,0.949550,0.966972,0.958182


Text: The economy is improving according to alien scientists
Prediction: True News ✅
------------------------------------------------------------
Text: King Salman is not the King of Saudi Arabia.
Prediction: Misinformation ❌
------------------------------------------------------------
Text: COVID-19 vaccines have been approved by the World Health Organization
Prediction: True News ✅
------------------------------------------------------------
Text: Aliens landed in New York and started dancing on Broadway.
Prediction: Misinformation ❌
------------------------------------------------------------
Text: 	The moon is made entirely of cheese
Prediction: Misinformation ❌
------------------------------------------------------------
Text: The Eiffel Tower is located in Paris, France
Prediction: Misinformation ❌
------------------------------------------------------------

📊 Final Evaluation Results on Test Set:
eval_loss: 0.1101
eval_accuracy: 0.9663
eval_precision: 0.9616
eval_recall: 0.9690